In [31]:
pip install streamlit openai mem0ai agno pyngrok cloudflared google-generativeai

In [ ]:
%%writefile app.py
import streamlit as st
from agno.agent import Agent
from agno.models.openai.like import OpenAILike
import google.generativeai as genai
from mem0 import MemoryClient

# === API KEYS ===
genai.configure(api_key="YOUR_GEMINI_API_KEY")  # Replace with your Gemini API Key
gemini = genai.GenerativeModel("gemini-1.5-flash")

client = MemoryClient(api_key="YOUR_MEM0_API_KEY")  # Replace with your Mem0 API Key

sutra_agent = Agent(
    model=OpenAILike(
        id="sutra-v2",
        api_key="YOUR_SUTRA-v2_API_KEY",  # Replace with your SUTRA API Key
        base_url="https://api.two.ai/v2"
    ),
    markdown=True,
    description="Multilingual medical assistant powered by SUTRA-V2",
    instructions=["Answer concisely in the requested language."]
)

# --- Streamlit App ---
st.title("Diabetes Bilingual Assistant 🤖🇮🇳")

if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []
if 'user_id' not in st.session_state:
    st.session_state.user_id = ""
if 'new_user' not in st.session_state:
    st.session_state.new_user = False

# --- User ID input ---
if st.session_state.user_id == "":
    with st.form("user_id_form"):
        user_id = st.text_input("Enter your unique User ID:")
        submitted = st.form_submit_button("Start Chat")
        if submitted:
            if user_id.strip() == "":
                st.warning("User ID cannot be empty.")
            else:
                st.session_state.user_id = user_id.strip()
                search_results = client.search("check", user_id=st.session_state.user_id)
                if len(search_results) == 0:
                    st.session_state.new_user = True
                st.rerun()

# --- New User Info Collection ---
if st.session_state.new_user:
    st.subheader("New User Information")
    with st.form("new_user_form"):
        name = st.text_input("Full Name / तुमचं पूर्ण नाव:")
        age = st.text_input("Age / वय:")
        diabetes_type = st.text_input("Type of Diabetes / डायबिटीजचा प्रकार:")
        medication = st.text_input("Medications / औषधे:")
        symptoms = st.text_input("Symptoms / लक्षणे:")
        location = st.text_input("Current Location / सध्याचं ठिकाण:")

        submitted = st.form_submit_button("Save Info")
        if submitted:
            messages = [
                {"role": "user", "content": f"My name is {name}."},
                {"role": "user", "content": f"I am {age} years old."},
                {"role": "user", "content": f"I have {diabetes_type} diabetes."},
                {"role": "user", "content": f"My medications include: {medication}."},
                {"role": "user", "content": f"My symptoms include: {symptoms}."},
                {"role": "user", "content": f"I currently live in: {location}."},
                {"role": "assistant", "content": f"Thanks {name}, your health info is stored for personalized support."}
            ]
            client.add(messages, user_id=st.session_state.user_id)
            st.success("Info saved! You can start chatting now.")
            st.session_state.new_user = False
            st.rerun()

# --- Chat Area ---
elif st.session_state.user_id != "":
    st.subheader("Chat with the Assistant")

    # Display chat history
    for chat in st.session_state.chat_history:
        st.write(f"**User ({chat['user_lang']}):** {chat['user_input']}")
        st.write(f"**Bot ({chat['bot_lang']}):** {chat['bot_output']}")

    with st.form("chat_form"):
        user_input = st.text_input("Your message:")
        submit = st.form_submit_button("Send")

        if submit and user_input:
            # Detect language
            detect_prompt = f"Detect language: {user_input} (only 'en' or 'mr')"
            response = sutra_agent.run(detect_prompt)
            detected_lang = response.content.strip()

            # Translate to English if Marathi
            if detected_lang == 'mr':
                translate_prompt = f"Translate to English: {user_input}"
                response = sutra_agent.run(translate_prompt)
                query_en = response.content.strip()
            else:
                query_en = user_input

            # === Proper Mem0 Context Retrieval & Personal Info Extraction ===
            search_results = client.search(query_en, user_id=st.session_state.user_id)

            mem0_memories = []
            for r in search_results:
                if 'message' in r and 'content' in r['message']:
                    role = r['message'].get('role', 'user').capitalize()
                    content = r['message']['content']
                    mem0_memories.append(f"{role}: {content}")
                elif 'memory' in r:
                    mem0_memories.append(f"Memory: {r['memory']}")

            context = "\n".join(mem0_memories)

            # Extract personal info properly
            user_name, diabetes_type, location = "User", "Not specified", "Not specified"
            for r in search_results:
                memory_text = ''
                if 'message' in r and 'content' in r['message']:
                    memory_text = r['message']['content'].lower()
                elif 'memory' in r:
                    memory_text = r['memory'].lower()

                if "name is" in memory_text:
                    try:
                        user_name = memory_text.split("name is")[-1].split()[0].capitalize()
                    except:
                        pass
                if "type 1" in memory_text or "type 2" in memory_text:
                    diabetes_type = "Type 2" if "type 2" in memory_text else "Type 1"
                if "from" in memory_text:
                    try:
                        location = memory_text.split("from")[-1].split(".")[0].strip().capitalize()
                    except:
                        pass

            # === Final Prompt Building ===
            if detected_lang == 'mr':
                final_prompt = f"""
                तुम्ही एक मधुमेह सहाय्यक आहात. {user_name} नावाच्या रुग्णाला {location} येथील {diabetes_type} मधुमेह आहे.

                पूर्वीच्या संभाषणातील आठवणी:
                {context}

                वापरकर्त्याचा नवीन प्रश्न:
                "{query_en}"

                मधुमेह-सुरक्षित आणि व्यावहारिक उत्तर द्या.
                जर रुग्ण भारतात असेल तर ते फक्त भारतीयांसाठीच करा.
                """
            else:
                final_prompt = f"""
                You are a diabetes-friendly AI assistant helping {user_name}, a patient from {location} diagnosed with {diabetes_type} Diabetes.

                Past memories from previous chats:
                {context}

                User's new query:
                "{query_en}"

                Provide a helpful, diabetes-safe, and practical response.
                Make it Indian-specific if the patient is in India.
                """

            # Gemini generates response
            gemini_response = gemini.generate_content(final_prompt)

            # Translate output to Marathi if needed
            if detected_lang == 'mr':
                translate_back_prompt = f"Translate this to Marathi: {gemini_response.text}"
                response_back = sutra_agent.run(translate_back_prompt)
                bot_output = response_back.content.strip()
                bot_lang = "mr"
            else:
                bot_output = gemini_response.text.strip()
                bot_lang = "en"

            # Show bot response
            st.session_state.chat_history.append({
                'user_input': user_input,
                'user_lang': detected_lang,
                'bot_output': bot_output,
                'bot_lang': bot_lang
            })

            # Save this interaction to Mem0
            memory_data = [
                {"role": "user", "content": user_input},
                {"role": "assistant", "content": bot_output}
            ]
            client.add(memory_data, user_id=st.session_state.user_id)

            st.rerun()

    # Exit button
    if st.button("Exit Chat"):
        st.session_state.user_id = ""
        st.session_state.chat_history = []
        st.session_state.new_user = False
        st.rerun()

Overwriting app.py


In [33]:
!streamlit run app.py & npx cloudflared tunnel --url http://localhost:8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.9.12.125:8501

⠴⠦⠧⠇2025-06-24T11:47:08Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-06-24T11:47:08Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-06-24T11:47:12Z INF +--------------------------------------------------------------------------------------------+
2025-06-24T11:47:12